In [ ]:
query = '''
SELECT *,
avg(result) OVER (PARTITION BY horse_id ORDER BY horse_id ASC, date ASC ROWS 3 PRECEDING EXCLUDE CURRENT ROW) AS Three_Race_Rolling_Avg_Finish,
avg(lengths_behind) OVER (PARTITION BY horse_id ORDER BY horse_id ASC, date ASC ROWS 3 PRECEDING EXCLUDE CURRENT ROW) AS Three_Race_Rolling_Average_Lengths,
avg(finish_time) OVER (PARTITION BY horse_id ORDER BY horse_id ASC, date ASC ROWS 3 PRECEDING EXCLUDE CURRENT ROW) AS Three_Race_Rolling_Average_Time,
avg(distance / finish_time) OVER (PARTITION BY horse_id ORDER BY horse_id ASC, date ASC ROWS 3 PRECEDING EXCLUDE CURRENT ROW) AS Three_Race_Rolling_Average_Distance_Per_time,
FROM combined_table_1
LIMIT 5
;
'''

Other metrics to add: 
- DONE Horse's Rating Compared to field:
    - avg_field_rating
    - diff_from_field_avg
    - field_rank (rename field_rating_rank)
- Shows Over last 10 races:
    - shows_in_last_ten_races
    - career_races
    - career_shows
- DONE Age Relative to Field:
    - avg_field_age
    - field_age_rank	
    - diff_from_field_age_avg
- DONE Horse's Weights Relative to the Field:
    - field_dec_wgt_rank	
    - avg_field_dec_wgts	
    - diff_from_field_declared_wgt_avg	
    - field_avg_handicap_wgt
    - diff_from_field_handicap_wgt_avg
    - field_handicap_wgt_rank

Horses Rating Compared to Field:

In [ ]:
query = '''
WITH race_ratings AS (
SELECT race_id, avg(horse_rating) AS avg_field_rating
FROM runs_table
GROUP BY race_id
)

SELECT ru.race_id, ru.result, ru.horse_id, ru.horse_rating, 
rr.avg_field_rating, 
rank() OVER my_window as field_rank, 
ru.horse_rating - rr.avg_field_rating AS diff_from_field_avg
FROM runs_table as ru
LEFT JOIN race_ratings AS rr
    ON ru.race_id = rr.race_id
WINDOW my_window AS (PARTITION BY ru.race_id ORDER BY ru.horse_rating DESC, ru.race_id DESC)
ORDER BY ru.race_id DESC, ru.result ASC
LIMIT 50
;
'''
df = pd.read_sql(query, engine)

df

Age Relative to Field:

In [ ]:
query = '''
WITH race_ages AS (
SELECT race_id, avg(horse_age) AS avg_field_age
FROM runs_table
GROUP BY race_id
)

SELECT ru.race_id, ru.horse_id, ru.result, ru.horse_age, 
ra.avg_field_age, 
rank() OVER my_window as field_age_rank, 
ru.horse_age - ra.avg_field_age AS diff_from_field_age_avg
FROM runs_table as ru
LEFT JOIN race_ages AS ra
    ON ru.race_id = ra.race_id
WINDOW my_window AS (PARTITION BY ru.race_id ORDER BY ru.horse_age DESC, ru.race_id DESC)
--ORDER BY ru.race_id DESC, ru.horse_id ASC
LIMIT 50
;
'''
df = pd.read_sql(query, engine)

df

Weights:

Ranking of the declared weight:

In [ ]:
query = '''
WITH race_wgts AS (
SELECT race_id, avg(declared_weight) AS avg_field_dec_wgts, avg(actual_weight) AS avg_actual_wgt
FROM runs_table
GROUP BY race_id
)

SELECT ru.race_id, ru.horse_id, ru.result, 
rank() OVER my_window_1 as field_dec_wgt_rank,
ru.declared_weight,  
ra.avg_field_dec_wgts, 
ru.declared_weight - ra.avg_field_dec_wgts AS diff_from_field_declared_wgt_avg,
ru.actual_weight AS handicap_wgt,
ra.avg_actual_wgt AS field_avg_handicap_wgt,
ru.actual_weight - ra.avg_actual_wgt AS diff_from_field_handicap_wgt_avg
FROM runs_table as ru
LEFT JOIN race_wgts AS ra
    ON ru.race_id = ra.race_id
WINDOW my_window_1 AS (PARTITION BY ru.race_id ORDER BY ru.declared_weight DESC, ru.race_id DESC)
ORDER BY ru.race_id DESC, ru.result ASC
LIMIT 50
;
'''

Ranking of the actual weight:

In [ ]:
query = '''
WITH race_wgts AS (
SELECT race_id, avg(declared_weight) AS avg_field_dec_wgts, avg(actual_weight) AS avg_actual_wgt
FROM runs_table
GROUP BY race_id
)

SELECT ru.race_id, ru.horse_id, ru.result, 
rank() OVER my_window_1 as field_handicap_wgt_rank,
ru.actual_weight AS handicap_wgt,
ra.avg_actual_wgt AS field_avg_handicap_wgt,
ru.actual_weight - ra.avg_actual_wgt AS diff_from_field_handicap_wgt_avg
FROM runs_table as ru
LEFT JOIN race_wgts AS ra
    ON ru.race_id = ra.race_id
WINDOW my_window_1 AS (PARTITION BY ru.race_id ORDER BY ru.actual_weight DESC, ru.race_id DESC)
ORDER BY ru.race_id DESC, ru.result ASC
LIMIT 50
;
'''
df = pd.read_sql(query, engine)

df

Shows:

## Shows Over Last 5 Races and Career Show Rate

In [1]:
query = '''
WITH shows_table AS (
SELECT ru.race_id, ru.horse_id, ru.result as race_result, ra.date,
CASE WHEN ru.result <= 3 THEN 1
ELSE 0 END AS Show
FROM runs_table as ru
LEFT JOIN races_table as ra
    ON ru.race_id = ra.race_id
)


SELECT race_id, horse_id, race_result, date, show,
(count(race_id) OVER (
PARTITION BY horse_id 
ORDER BY date ASC ROWS UNBOUNDED PRECEDING EXCLUDE CURRENT ROW)) AS career_races, 
(sum(show) OVER (
PARTITION BY horse_id 
ORDER BY date ASC ROWS UNBOUNDED PRECEDING EXCLUDE CURRENT ROW)) AS career_shows,
SUM(Show) OVER (
PARTITION BY horse_id 
ORDER BY horse_id ASC, date ASC ROWS 5 PRECEDING EXCLUDE CURRENT ROW) AS Shows_in_Last_5_races
FROM shows_table
ORDER BY horse_id ASC
LIMIT 50
;
'''
df = pd.read_sql(query, engine)

df

NameError: name 'pd' is not defined

Above will tell you how many Shows the horse has had in the 10 preceding races.  Again, each horse's first race will have a NaN.

### Career Show Rate:

Calculating the rate at which a horse shows, prior to each race:

In [13]:
query = '''
WITH shows_table AS (
SELECT ru.race_id, ru.horse_id, ru.result as race_result, ra.date,
CASE WHEN ru.result <= 3 THEN 1
ELSE 0 END AS Show
FROM runs_table as ru
LEFT JOIN races_table as ra
    ON ru.race_id = ra.race_id
)


SELECT race_id, horse_id, race_result, date, show,
(count(race_id) OVER (
PARTITION BY horse_id 
ORDER BY date ASC ROWS UNBOUNDED PRECEDING EXCLUDE CURRENT ROW)) AS career_races, 
(sum(show) OVER (
PARTITION BY horse_id 
ORDER BY date ASC ROWS UNBOUNDED PRECEDING EXCLUDE CURRENT ROW)) AS career_shows
FROM shows_table
ORDER BY horse_id ASC
LIMIT 50
;
'''
df = pd.read_sql(query, engine)

df

,race_id,horse_id,race_result,date,show,career_races,career_shows
0,3396,0,10,2001-11-04,0,0,NaN
1,3489,0,9,2001-12-09,0,1,0.0
2,3794,0,14,2002-06-02,0,2,0.0
3,3835,0,12,2002-06-23,0,3,0.0
4,3886,0,4,2002-07-14,0,4,0.0
5,3944,0,10,2002-08-04,0,5,0.0
6,4008,0,8,2002-08-28,0,6,0.0
7,4037,0,3,2002-09-07,1,7,0.0
8,4112,0,2,2002-10-06,1,8,1.0
9,4177,0,14,2002-11-03,0,9,2.0


Rate could easily be calculated here - however each first race would have an NaN.